In [1]:
using GLPK
using LinearAlgebra
using MathOptInterface
const MOI = MathOptInterface
using Random
using DelimitedFiles

In [2]:
row_norm(X) = [norm(X[i, :]) for i in 1:size(X)[1]];
norm_2_0(X) = norm(row_norm(X), 0);

## Loading Data

In [3]:
totaltime, totallines = open("T1/R4T1.txt") do f
    global lines = readlines(f);
end

69367-element Vector{String}:
 "matrix,row,col"
 "S,2172,4456"
 "I,J,V"
 "1.0,1.0,-4.0"
 "4.0,1.0,3.0"
 "8.0,1.0,1.0"
 "12.0,1.0,1.0"
 "15.0,1.0,-1.0"
 "27.0,1.0,-1.0"
 "3.0,2.0,1.0"
 "7.0,2.0,-1.0"
 "14.0,2.0,2.0"
 "40.0,2.0,-1.0"
 ⋮
 "4361.0,50.0,0.8734623588776002"
 "4362.0,50.0,-1.0711041643503127"
 "4382.0,50.0,-0.24154267262532617"
 "4410.0,50.0,-0.18451477102981703"
 "4413.0,50.0,-0.19879609752002775"
 "4415.0,50.0,0.007816902279073563"
 "4424.0,50.0,6.4842004556170545e-19"
 "4429.0,50.0,-1.9601377265185305"
 "4435.0,50.0,-0.007269663698619032"
 "4442.0,50.0,8.989261888210545e-17"
 "4452.0,50.0,-1.7826464834011574e-17"
 "4455.0,50.0,-0.9641332790949555"

In [4]:
function is_info_line(a_line)
    a_line_info = split(a_line, ",");
    line_indicator = a_line_info[1];
    if cmp(line_indicator, "matrix") == 0 || cmp(line_indicator, "vector") == 0
        return true
    end
    return false
end

is_info_line (generic function with 1 method)

In [5]:
function get_info_end_line(lines_info, t_start, t_end_file)
    current_line = lines_info[t_start];
    t = t_start;
    for t in t_start:t_end_file
        current_line = lines_info[t];
        if is_info_line(current_line)
            return t-1
        end
    end
    return t_end_file
end

get_info_end_line (generic function with 1 method)

In [6]:
function fill_matrix(A, n_rows, n_cols, values_vector)
    for info_line in values_vector
        line_info = split(info_line, ",");
        i_v = floor(Int64, parse(Float64, line_info[1]));
        j_v = floor(Int64, parse(Float64, line_info[2]));
        val = parse(Float64, line_info[3]);
        A[i_v, j_v] = val;
    end
end

fill_matrix (generic function with 1 method)

In [7]:
function read_block(t_start)
    info_line = lines[t_start];
    info = split(info_line, ",");
    t = t_start + 1;
    
    if cmp(info[1],"matrix") == 0
        matrix_info_line = lines[t];
        matrix_info = split(matrix_info_line, ",");
        matrix_name = matrix_info[1];
        num_rows = parse(Int64, matrix_info[2]);
        num_cols = parse(Int64, matrix_info[3]);
        matrix_kind = 0;
        if cmp(matrix_name, "S") == 0
            global m = num_rows;
            global n = num_cols;
            global S = zeros(m, n);
            matrix_kind = 1;
        elseif cmp(matrix_name, "L") == 0
            global c = num_cols;
            global L = zeros(n, c);
            matrix_kind = 2;
        elseif cmp(matrix_name, "U") == 0
            global c = num_cols;
            global U = zeros(n, c);
            matrix_kind = 3;
        else
            @show "Something is not right here! - id:1"
        end
        t = t+1;
        IJV_line = lines[t];
        if cmp(IJV_line, "I,J,V") != 0
            @show "Something is not right here! - id:2"
        end
        t = t+1;
        t_last = get_info_end_line(lines, t, file_num_lines);
        if matrix_kind == 1
            fill_matrix(S, m, n, lines[t:t_last]);
        elseif matrix_kind == 2
            fill_matrix(L, n, c, lines[t:t_last]);
        elseif matrix_kind == 3
            fill_matrix(U, n, c, lines[t:t_last]);
        end
        t = t_last+1;
    elseif cmp(info[1],"vector") == 0
        vector_info_line = lines[t];
        if cmp(vector_info_line, "lambda,1,1") != 0
            @show "Something is not right here! - 3"
        end
        t = t+1;
        IV_line = lines[t];
        if cmp(IV_line, "I,V") != 0
            @show "Something is not right here! - 4"
        end
        t = t+1;
        lambda_info_line = lines[t];
        lambda_info = split(lambda_info_line, ",");
        global lambda = parse(Float64, lambda_info[2]);
        t = t+1;
    else
        @show "Something is not right here! - 5"
    end
    return t
end

read_block (generic function with 1 method)

In [8]:
file_num_lines = length(lines)
m = 0;
n = 0;
c = 0;
lambda = 0;
line_counter = 1;
while line_counter < file_num_lines
    line_counter = read_block(line_counter);
end

In [9]:
lambda

133.68

### Get Intuition From Data:

In [10]:
function revise_resolution(M, res)
    num_rows, num_cols = size(M);
    Mp = copy(M);
    for i in 1:num_rows
        for j in 1:num_cols
            if abs(M[i,j]) < res
                Mp[i,j] = 0;
            end
        end
    end
    return Mp
end

revise_resolution (generic function with 1 method)

In [11]:
L_tilde = revise_resolution(L, 2e-5);
U_tilde = revise_resolution(U, 2e-5);

In [12]:
function get_nonzero_indeces(L, U)
    nonzero_ind = zeros(Int64, 0);
    zero_indeces = zeros(Int64, 0);
    for i in 1:n
        if maximum(L[i, :])>0 || minimum(U[i, :])<0
            append!(nonzero_ind, i);
        else
            append!(zero_indeces, i);
        end
    end
    return nonzero_ind, zero_indeces
end

get_nonzero_indeces (generic function with 1 method)

In [13]:
nz_ind, z_ind = get_nonzero_indeces(L_tilde, U_tilde);

In [14]:
function get_trivial_indeces(l, u)
    trivial_indeces = zeros(Int64, 0);
    non_trivial_indeces = zeros(Int64, 0);
    for i in 1:n
        if l[i]==u[i]
            append!(trivial_indeces, i);
        else
            append!(non_trivial_indeces, i);
        end
    end
    return trivial_indeces, non_trivial_indeces
end

get_trivial_indeces (generic function with 1 method)

In [15]:
function make_full_v0(vec_hat, non_trivial_ind, n_h)
    v0_full = zeros(n);
    for ind in 1:n_h
        i = non_trivial_ind[ind];
        v0_full[i] = vec_hat[ind];
    end
    return v0_full
end

make_full_v0 (generic function with 1 method)

## Part 4

$$
minimize  \quad (\left\lVert V \right\rVert_{2,0}, \left\lVert (SV)^T \right\rVert_{2,0}) \\
s.t.: \quad L \preceq V \preceq U
$$ 
by using: 
$$
minimize  \quad \left\lVert V \right\rVert_{1,1} + \lambda \left\lVert (SV)^T \right\rVert_{1,1} \\
s.t.: \quad L \preceq V \preceq U
$$

### L1,1 Approximation:

#### All Sv = 0:

In [30]:
V0 = zeros((n,c));
for col in 1:c
    triv_ind, non_triv_ind = get_trivial_indeces(L_tilde[:,col], U_tilde[:, col]);
    n_hat = length(non_triv_ind);
    S_hat = S[:, non_triv_ind];
    l_hat = L_tilde[non_triv_ind, col];
    u_hat = U_tilde[non_triv_ind, col];
    obective_coeffs = ones(n_hat);
    for i in 1:n_hat
        if non_triv_ind[i] in nz_ind
            obective_coeffs[i] = 0;
        end
    end
    # ------------------------------------------------------------------------------------
    optimizer = GLPK.Optimizer()
    v_hat = MOI.add_variables(optimizer, n_hat)
    v_abs_hat = MOI.add_variables(optimizer, n_hat)
    # ------------------------------------------------------------------------------------
    objective_function = MOI.ScalarAffineFunction(MOI.ScalarAffineTerm.(obective_coeffs, v_abs_hat), 0.0)
    MOI.set(optimizer, 
            MOI.ObjectiveFunction{MOI.ScalarAffineFunction{Float64}}(),
            objective_function)
    MOI.set(optimizer, MOI.ObjectiveSense(), MOI.MIN_SENSE)
    # ------------------------------------------------------------------------------------
    for i in 1:n_hat
            MOI.add_constraint(optimizer, 
                               MOI.SingleVariable(v_hat[i]), 
                               MOI.Interval(l_hat[i], u_hat[i]))
    end
    # ------------------------------------------------------------------------------------
    for (i,row) in enumerate(eachrow(S_hat))
            row_function = MOI.ScalarAffineFunction(MOI.ScalarAffineTerm.(row, v_hat), 0.0);
            MOI.add_constraint(optimizer, row_function, MOI.EqualTo(0.0))
    end
    # ------------------------------------------------------------------------------------
    for i in 1:n_hat
            abs_plus = MOI.ScalarAffineFunction(MOI.ScalarAffineTerm.([1.0, -1.0], [v_abs_hat[i], v_hat[i]]), 0.0)
            abs_minus = MOI.ScalarAffineFunction(MOI.ScalarAffineTerm.([1.0, 1.0], [v_abs_hat[i], v_hat[i]]), 0.0)
            MOI.add_constraint(optimizer, 
                               abs_plus, 
                               MOI.GreaterThan(0.0))
            MOI.add_constraint(optimizer, 
                               abs_minus, 
                               MOI.GreaterThan(0.0))
    end
    # ------------------------------------------------------------------------------------
    MOI.optimize!(optimizer)
    status = MOI.get(optimizer, MOI.TerminationStatus())
    if status != MOI.OPTIMAL
        @show "Somethings wrong here"
    end
    v0_hat = MOI.get(optimizer, MOI.VariablePrimal(), v_hat)
    # ------------------------------------------------------------------------------------
    v0 = make_full_v0(v0_hat, non_triv_ind, n_hat);
    V0[:,col] = v0;
end

In [31]:
V0 = (abs.(V0).>1e-6).*V0;
norm_2_0(V0)

699.0

#### All Sv != 0:

In [32]:
V1 = zeros((n,c));
for col in 1:c
    triv_ind, non_triv_ind = get_trivial_indeces(L_tilde[:,col], U_tilde[:, col]);
    n_hat = length(non_triv_ind);
    S_hat = S[:, non_triv_ind];
    l_hat = L_tilde[non_triv_ind, col];
    u_hat = U_tilde[non_triv_ind, col];
    obective_coeffs = ones(n_hat);
    for i in 1:n_hat
        if non_triv_ind[i] in nz_ind
            obective_coeffs[i] = 0;
        end
    end
    # ------------------------------------------------------------------------------------
    optimizer = GLPK.Optimizer()
    v_hat = MOI.add_variables(optimizer, n_hat)
    v_abs_hat = MOI.add_variables(optimizer, n_hat)
    # ------------------------------------------------------------------------------------
    objective_function = MOI.ScalarAffineFunction(MOI.ScalarAffineTerm.(obective_coeffs, v_abs_hat), 0.0)
    MOI.set(optimizer, 
            MOI.ObjectiveFunction{MOI.ScalarAffineFunction{Float64}}(),
            objective_function)
    MOI.set(optimizer, MOI.ObjectiveSense(), MOI.MIN_SENSE)
    # ------------------------------------------------------------------------------------
    for i in 1:n_hat
            MOI.add_constraint(optimizer, 
                               MOI.SingleVariable(v_hat[i]), 
                               MOI.Interval(l_hat[i], u_hat[i]))
    end
    # ------------------------------------------------------------------------------------
    for i in 1:n_hat
            abs_plus = MOI.ScalarAffineFunction(MOI.ScalarAffineTerm.([1.0, -1.0], [v_abs_hat[i], v_hat[i]]), 0.0)
            abs_minus = MOI.ScalarAffineFunction(MOI.ScalarAffineTerm.([1.0, 1.0], [v_abs_hat[i], v_hat[i]]), 0.0)
            MOI.add_constraint(optimizer, 
                               abs_plus, 
                               MOI.GreaterThan(0.0))
            MOI.add_constraint(optimizer, 
                               abs_minus, 
                               MOI.GreaterThan(0.0))
    end
    # ------------------------------------------------------------------------------------
    MOI.optimize!(optimizer)
    status = MOI.get(optimizer, MOI.TerminationStatus())
    if status != MOI.OPTIMAL
        @show "Somethings wrong here"
    end
    v0_hat = MOI.get(optimizer, MOI.VariablePrimal(), v_hat)
    # ------------------------------------------------------------------------------------
    v0 = make_full_v0(v0_hat, non_triv_ind, n_hat);
    V1[:,col] = v0;
end

In [33]:
V1 = (abs.(V1).>1e-6).*V1;
norm_2_0(V1)

181.0

### Checking which constraints are usefull:

In [34]:
distances = zeros(c);
for col in 1:c
    distances[col] = norm(V0[:,col], 1) - norm(V1[:,col], 1);
end

In [35]:
SV_flag = zeros(Int64, c);
for col in 1:c
    if distances[col] < lambda
        SV_flag[col] = 1;
    end
end

### Weighted Algorithm for swiched SVs

In [36]:
eps = 1e-4;
p = 1;
NUM_RUNS = 20;

In [37]:
function W4(Val0, is_rand)
    W = ones(n);
    for i in 1:n
        s = norm(Val0[i,:] ,2) + eps;
        W[i] = (1+s^p)/(s^(p+1));
        if is_rand
            W[i] = W[i]*rand()^3;
        end
    end
    return W
end

W4 (generic function with 1 method)

In [38]:
best_nnz = norm_2_0(V0);
best_V = V0;
W = W4(V0, true);

for k in 1:NUM_RUNS  
    V0 = zeros((n,c));
    for col in 1:c
        triv_ind, non_triv_ind = get_trivial_indeces(L_tilde[:,col], U_tilde[:, col]);
        n_hat = length(non_triv_ind);
        S_hat = S[:, non_triv_ind];
        l_hat = L_tilde[non_triv_ind, col];
        u_hat = U_tilde[non_triv_ind, col];
        obective_coeffs = ones(n_hat);
        for i in 1:n_hat
            if non_triv_ind[i] in nz_ind
                obective_coeffs[i] = 0;
            end
        end
        obective_weights = obective_coeffs .* W[non_triv_ind];
        # ------------------------------------------------------------------------------------
        optimizer = GLPK.Optimizer()
        v_hat = MOI.add_variables(optimizer, n_hat)
        v_abs_hat = MOI.add_variables(optimizer, n_hat)
        # ------------------------------------------------------------------------------------
        objective_function = MOI.ScalarAffineFunction(MOI.ScalarAffineTerm.(obective_weights, v_abs_hat), 0.0)
        MOI.set(optimizer, 
                MOI.ObjectiveFunction{MOI.ScalarAffineFunction{Float64}}(),
                objective_function)
        MOI.set(optimizer, MOI.ObjectiveSense(), MOI.MIN_SENSE)
        # ------------------------------------------------------------------------------------
        for i in 1:n_hat
                MOI.add_constraint(optimizer, 
                                   MOI.SingleVariable(v_hat[i]), 
                                   MOI.Interval(l_hat[i], u_hat[i]))
        end
        # ------------------------------------------------------------------------------------
        if SV_flag[col] == 1 
            for (i,row) in enumerate(eachrow(S_hat))
                    row_function = MOI.ScalarAffineFunction(MOI.ScalarAffineTerm.(row, v_hat), 0.0);
                    MOI.add_constraint(optimizer, row_function, MOI.EqualTo(0.0))
            end
        end
        # ------------------------------------------------------------------------------------
        for i in 1:n_hat
                abs_plus = MOI.ScalarAffineFunction(MOI.ScalarAffineTerm.([1.0, -1.0], [v_abs_hat[i], v_hat[i]]), 0.0)
                abs_minus = MOI.ScalarAffineFunction(MOI.ScalarAffineTerm.([1.0, 1.0], [v_abs_hat[i], v_hat[i]]), 0.0)
                MOI.add_constraint(optimizer, 
                                   abs_plus, 
                                   MOI.GreaterThan(0.0))
                MOI.add_constraint(optimizer, 
                                   abs_minus, 
                                   MOI.GreaterThan(0.0))
        end
        # ------------------------------------------------------------------------------------
        MOI.optimize!(optimizer)
        status = MOI.get(optimizer, MOI.TerminationStatus())
        if status != MOI.OPTIMAL
            @show "Somethings wrong here"
        end
        v0_hat = MOI.get(optimizer, MOI.VariablePrimal(), v_hat)
        # ------------------------------------------------------------------------------------
        v0 = make_full_v0(v0_hat, non_triv_ind, n_hat);
        V0[:,col] = v0;
    end
    V0 = (abs.(V0).>1e-6).*V0;
    @show norm_2_0(V0)

    nnz = norm_2_0(V0);
    if nnz < best_nnz
        best_nnz = nnz;
        best_V = V0;
    end

    # Adjust the weights elementwise and re-iterate
    W = W4(V0, true);
end

norm_2_0(V0) = 614.0
norm_2_0(V0) = 606.0
norm_2_0(V0) = 603.0
norm_2_0(V0) = 598.0
norm_2_0(V0) = 599.0
norm_2_0(V0) = 597.0
norm_2_0(V0) = 593.0
norm_2_0(V0) = 593.0
norm_2_0(V0) = 593.0
norm_2_0(V0) = 593.0
norm_2_0(V0) = 592.0
norm_2_0(V0) = 592.0
norm_2_0(V0) = 592.0
norm_2_0(V0) = 594.0
norm_2_0(V0) = 593.0
norm_2_0(V0) = 593.0
norm_2_0(V0) = 592.0
norm_2_0(V0) = 591.0
norm_2_0(V0) = 591.0
norm_2_0(V0) = 591.0


In [39]:
best_nnz
norm_2_0(best_V)

591.0

## Saving

In [209]:
f = open("T1/output.txt", "w") do f
  writedlm(f, best_V, ',')
end